In [ ]:
%cd drive/MyDrive/sqlovaa/sqlova

/content/drive/MyDrive/sqlovaa/sqlova


In [ ]:
!pip install flask-ngrok

In [ ]:
!pip3 install pyrebase

In [ ]:
!pip3 install sqlalchemy==1.3
!pip3 install records==0.5.2
!pip3 install babel matplotlib defusedxml tqdm
!pip3 install ujson stanza

import stanza
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
#stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

!ls $CORENLP_HOME

     |████████████████████████████████| 5.9 MB 15.4 MB/s 
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.0-cp37-cp37m-linux_x86_64.whl size=1182808 sha256=6978f31c705179113afd7c489ede455a4dd18230e72a658096f4e2a72d0f6c57
  Stored in directory: /root/.cache/pip/wheels/cb/b4/c8/1ae5d5a576b48cf3ea64943280034378cd76055c0f207bc4c1
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.25
    Uninstalling SQLAlchemy-1.4.25:
      Successfully uninstalled SQLAlchemy-1.4.25
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 214 kB 12.8 MB/s 
     |████████████████████████████████| 432 kB 35.0 MB/s 
     |████████████████████████████████| 168 kB 72.8 MB/s 
  Created wheel for emoji: filename=emoji-1.6.0-py3-none-any.whl size=168256 sha256=3c702eee65e09fa87c91e83680cea7e28d380739485acc3a404556bf027c549a
  Stored in directory: /root/.cache/pip/wheels/f7/d7/74/c720aaf345a042b0c2d743618732

In [ ]:
import os, json

from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template

import pyrebase

import add_question_ale
from add_question_ale import question_to_json
import add_csv_ale
from add_csv_ale import csv_to_sqlite,csv_to_json
import annotate_ws
from annotate_ws import annotate_example_ws, count_lines
import predict_ale
from predict_ale import run_predict

from tqdm import tqdm

import jsonl

config = {
    "apiKey": "---------------------",
    "authDomain": "---------------------",
    "databaseURL": "---------------------",
    "projectId": "---------------------",
    "storageBucket": "---------------------",
    "messagingSenderId": "---------------------",
    "appId": "---------------------",
    "measurementId": "---------------------"
}

firebase = pyrebase.initialize_app(config)

db = firebase.database()
storage = firebase.storage()

app = Flask(__name__)
run_with_ngrok(app)   

user_id=""

def annotate(din, dout, user_id):
    fquestions = os.path.join(din, user_id) + '.jsonl'
    ftable = os.path.join(din, user_id) + '.tables.jsonl'
    ftok = os.path.join(dout, user_id) + '_tok.jsonl'
    
    with open(fquestions) as fq, open(ftable) as ft, open(ftok, 'wt') as fo:
            
            tables = {}
            for line in tqdm(ft, total=count_lines(ftable)):
                d = json.loads(line)
                tables[d['id']] = d
            print('loading examples')
            n_written = 0
            cnt = -1
            for line in tqdm(fq, total=count_lines(fquestions)):
                cnt += 1
                d = json.loads(line)
                a = annotate_example_ws(d, tables[d['table_id']])
                fo.write(json.dumps(a) + '\n')
                n_written += 1

def print_predicted(dsaved, user_id):
    origin_data = jsonl.load_jsonl("{}/results_{}.jsonl".format(dsaved,user_id))
    nlu_sql={}
    for d in origin_data:
        nlu_sql[d['nlu']]=d['sql']
    return nlu_sql


# route main page
@app.route('/')
def index():
    return render_template("index3.html")

# data prediction
@app.route('/add_data', methods=['POST'])
#@app.route('/', methods=['POST'])
def add_data():
    if request.method =='POST':
        #upload data
        user_id = request.form['userid']
        #tid = request.form['tid']
        dsaved = './data_and_model/flask_test'
        json_file_name = '{}.jsonl'.format(user_id)

        table = request.files['table']
        table_name = table.filename
        tid = table_name[:-4]
        table.save(os.path.join(dsaved, table_name))
        
        query = request.form['query']
        questions = query.split('\n')

        for q in questions:
          question_to_json(tid, q.strip('\n\r'), json_file_name, dsaved)

        storage.child("{}/{}".format(user_id, json_file_name)).put("{}/{}".format(dsaved, json_file_name))

        #storage.child("tables/{}.csv".format(tid)).put(table)
        
        csv_to_sqlite(tid, table_name, '{}.db'.format(user_id), dsaved)
        csv_to_json(tid, table_name, '{}.tables.jsonl'.format(user_id), dsaved)
        storage.child("{}/{}.db".format(user_id, user_id)).put("{}/{}.db".format(dsaved, user_id))
        storage.child("{}/{}.tables.jsonl".format(user_id, user_id)).put("{}/{}.tables.jsonl".format(dsaved, user_id))
        os.remove(os.path.join(dsaved, table_name))

        annotate(dsaved, dsaved, user_id)
        storage.child("{}/{}_tok.jsonl".format(user_id, user_id)).put("{}/{}_tok.jsonl".format(dsaved, user_id))
        #return render_template("index3.html", predicted = predicted)
        
        ###run_predict(model_file, bert_model_file, bert_path, data_path, user_id, result_path)
        run_predict('./data_and_model/model_best.pt', './data_and_model/model_bert_best.pt', './data_and_model', dsaved, user_id,'./data_and_model/flask_test/')

        result_dict = print_predicted(dsaved, user_id)
        return render_template("predicted.html", predicted=result_dict.items())
    return render_template("index3.html")

if __name__=='__main__':
	app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7df0-34-90-59-171.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Oct/2021 19:59:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2021 19:59:04] "GET /favicon.ico HTTP/1.1" 404 -
100%|██████████| 1/1 [00:00<00:00, 2878.73it/s]


loading examples


...

100%|██████████| 2/2 [00:38<00:00, 19.23s/it]

<class 'CoreNLP_pb2.Document'>


Batch_size = 32
BERT parameters:
learning rate: 1e-05
Fine-tune BERT: False
vocab size: 30522
hidden_size: 1024
num_hidden_layer: 24
num_attention_heads: 16
hidden_act: gelu
intermediate_size: 4096
hidden_dropout_prob: 0.1
attention_probs_dropout_prob: 0.1
max_position_embeddings: 512
type_vocab_size: 2
initializer_range: 0.02
Seq-to-SQL: the number of final BERT layers to be used: 2
Seq-to-SQL: the size of hidden dimension = 100
Seq-to-SQL: LSTM encoding layer size = 2
Seq-to-SQL: dropout rate = 0.3
Seq-to-SQL: learning rate = 0.001


127.0.0.1 - - [12/Oct/2021 20:00:08] "POST /add_data HTTP/1.1" 200 -


{"query": {"agg": 0, "sel": 1, "conds": [[0, 0, "ch-47d chinook"]]}, "table_id": "1111111", "nlu": "What if the description of a ch-47d chinook?", "sql": "SELECT (Description) FROM 1111111 WHERE Aircraft = ch-47d chinook"}

1
{"query": {"agg": 0, "sel": 2, "conds": [[0, 0, "robinson r"]]}, "table_id": "1111111", "nlu": "What is the max gross weight of the Robinson R-22?", "sql": "SELECT (Max Gross Weight) FROM 1111111 WHERE Aircraft = robinson r"}

2
Loaded 2 records from ./data_and_model/flask_test/results_ssom.jsonl
